In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# URL to scrape
base_url = "https://ask.herts.ac.uk/academic-support"

# Send a request to the URL
response = requests.get(base_url)
content = response.content

# Parse the HTML content
soup = BeautifulSoup(content, "html.parser")

# Function to extract and format text with links
def extract_text_with_links(soup, base_url):
    elements = soup.find_all(['p', 'h2', 'h3', 'li'])
    text_with_links = []
    for element in elements:
        links = element.find_all('a', href=True)
        if links:
            for link in links:
                link_text = link.get_text(strip=True)
                link_url = urljoin(base_url, link['href'])
                link.replace_with(f"{link_text} [{link_url}]")
        text_with_links.append(element.get_text(strip=True))
    return "\n".join(text_with_links)

# Extract and clean text with embedded links
extracted_text = extract_text_with_links(soup, base_url)
print(extracted_text)


HOME [https://ask.herts.ac.uk/]
SEARCH [https://ask.herts.ac.uk/search]
CATEGORIES [https://ask.herts.ac.uk/academic-support]FeaturedAsk Herts Services [https://ask.herts.ac.uk/browse/student-information/student-information-service]Exams and Assessments [https://ask.herts.ac.uk/browse/student-information/exams-and-assessments]Fees, Funding and Finance [https://ask.herts.ac.uk/browse/student-information/fees-funding-and-finance]Finding Employment [https://ask.herts.ac.uk/browse/student-jobs-careers-and-skills/finding-employment]Food and Drink [https://ask.herts.ac.uk/browse/student-community/food-and-drink]Getting Started [https://ask.herts.ac.uk/browse/student-information/getting-started]Housing [https://ask.herts.ac.uk/browse/student-community/housing-and-accomodation]Travel, Parking and Directions [https://ask.herts.ac.uk/browse/student-information/travel-parking-and-directions]Voting in UK Elections [https://ask.herts.ac.uk/browse/student-voice/voting-in-uk-elections]InformationAcad

In [2]:
import requests
from bs4 import BeautifulSoup

# URL to scrape
base_url = "https://ask.herts.ac.uk/academic-support"

# Send a request to the URL
response = requests.get(base_url)
content = response.content

# Parse the HTML content
soup = BeautifulSoup(content, "html.parser")

# Print the entire HTML content
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7 "> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8 "> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9 "> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Academic support | Ask Herts | University of Hertfordshire
  </title>
  <meta content="The simple way for students at the University of Hertfordshire to find answers to your questions" name="description"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="app-id=1028074830" name="apple-itunes-app"/>
  <link href="/assets/webappicons/icon-60-6b7289e0dbd17cfc87047b719e80359b03bd625a2cda91748ed9389b58b96199.png" rel="apple-touch-icon"/>
  <link href="/assets/webappicons/icon-76-9345126bd6371128c6a3e11be02d83e946af43c63172ed4c786367a9fe2bda26.png" rel="apple-to

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json

# URL to scrape
base_url = "https://ask.herts.ac.uk/academic-support"

# Send a request to the URL
response = requests.get(base_url)
content = response.content

# Parse the HTML content
soup = BeautifulSoup(content, "html.parser")

# Function to extract and format text with links
def extract_text_with_links(soup, base_url):
    sections = []
    main_content = soup.find('div', {'id': 'main-content'})
    if not main_content:
        print("No content found with id 'main-content'")
        return sections
    
    current_section = None
    for element in main_content.find_all(['h2', 'h3', 'h4', 'p', 'li']):
        if element.name in ['h2', 'h3', 'h4']:
            if current_section:
                sections.append(current_section)
            headline = element.get_text(strip=True)
            current_section = {'headline': headline, 'content': ""}
        else:
            if current_section is None:
                current_section = {'headline': '', 'content': ""}
            text = element.get_text(strip=True)
            links = element.find_all('a', href=True)
            for link in links:
                link_text = link.get_text(strip=True)
                link_url = urljoin(base_url, link['href'])
                text = text.replace(link_text, f"{link_text} [{link_url}]")
            current_section['content'] += " " + text if current_section['content'] else text
    
    if current_section:
        sections.append(current_section)
    
    return sections

# Extract and clean text with embedded links
structured_content = extract_text_with_links(soup, base_url)

# Check if anything is extracted
if not structured_content:
    print("No content extracted")
else:
    # Save the structured content to a JSON file
    data = {"url": base_url, "content": structured_content}
    with open('AcademicSupport.json', 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)
    print("Content saved to AcademicSupport.json")


Content saved to AcademicSupport.json


In [4]:
import json

# Load the extracted structured content
with open('AcademicSupport.json', 'r', encoding='utf-8') as file:
    structured_content = json.load(file)["content"]

# Initialize the chatbot training structure
chatbot_data = {"intents": []}

# Define common intents
common_intents = {
    "greetings": {
        "patterns": [
            "Hello", "Hi", "Hey", "Good morning", "Good afternoon", "Good evening", "Hello there!",
            "Hiya!", "Hey!", "Morning!", "Afternoon!", "Evening!", "Greetings!", "What's up?", "Howdy!"
        ],
        "responses": [
            "Hello! How can I assist you today?",
            "Hi there! What can I help you with?",
            "Hey! How can I help?",
            "Good day! What do you need help with?"
        ]
    }
}

# Add common intents
for tag, data in common_intents.items():
    chatbot_data["intents"].append({
        "tag": tag,
        "patterns": data["patterns"],
        "responses": data["responses"]
    })

# Add extracted data as intents
for section in structured_content:
    tag = section["headline"]
    patterns = [
        f"What can you tell me about {tag}?",
        f"Tell me about {tag}",
        f"Give me information on {tag}",
        f"What is {tag}?",
        f"Explain {tag}",
        f"Details on {tag} please"
    ]
    responses = [section["content"]]
    
    chatbot_data["intents"].append({
        "tag": tag,
        "patterns": patterns,
        "responses": responses
    })

# Save the chatbot training data to a JSON file
with open('AcademicSupport_training.json', 'w', encoding='utf-8') as file:
    json.dump(chatbot_data, file, indent=4, ensure_ascii=False)

print("Chatbot training data saved to AcademicSupport_training.json")


Chatbot training data saved to AcademicSupport_training.json
